In [18]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [20]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(list_hh, i))
    ides.append(preprocessed_ide(list_hh, i))
    if i < 5:
        dialogues.append(preprocessed_dialogue(list_woz, i))
        ides.append(preprocessed_ide(list_woz, i))


In [24]:
dialogue_input, dialogue_output = sort(dialogues)
ide_input, ide_output = sort(ides)
maxLength = 0
# 21 is the max length
for i in dialogue_input:
    maxLength = max(maxLength, len(i))


In [25]:
one_hot_encoded_dialogue_input, one_hot_encoded_dialogue_output = np.array([]), np.array([])
sample = []
# Trying padding and processing
for i in dialogue_input:
    temp = i
    while (len(temp) < 21): temp.append(0)
    sample.append(temp)
    
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# sample = scaler.fit_transform(sample)
print(len(one_hot_encoded_dialogue_output))
one_hot_encoded_dialogue_input = tf.keras.utils.to_categorical(sample)

for i in dialogue_output:
    temp = tf.keras.utils.to_categorical(i).tolist()
    while (len(temp) < 47):
        temp.append(0)
    one_hot_encoded_dialogue_output = np.append(one_hot_encoded_dialogue_output, temp)
print(len(one_hot_encoded_dialogue_input), len(dialogue_output))

0
2808 2808


In [45]:
from keras import backend as K
from keras import metrics




def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
#     add f1 and/or loss function for the metric because of the state imbalance
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=10):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_train = y_train.reshape(-1)
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=2)
        scores.append(score[1])

    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = one_hot_encoded_dialogue_input
y = np.array(dialogue_output)

model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)

# confusion matrix- tells you 
# hand validation - write code that takes one hot encoding 


ValueError: in user code:

    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, s

In [14]:
from

65/65 [==============================] - 0s 1ms/step


ValueError: Classification metrics can't handle a mix of multiclass and unknown targets